In [ ]:
import pandas as pd

In [21]:
# DOR Monthly Data

from pathlib import Path

base = Path(r"J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Monthly Operating Statements")

year_dirs = [p for p in base.iterdir() if p.is_dir() and p.name.isdigit()]
if not year_dirs:
    raise FileNotFoundError(f"No year folders found under {base!s}")
latest_year_dir = max(year_dirs, key=lambda p: int(p.name))

pattern_files = list(
    latest_year_dir.glob("Cumulative Report - Operating Statements - *.xlsx")
)
xlsx_files = pattern_files or list(latest_year_dir.glob("*.xlsx"))
if not xlsx_files:
    raise FileNotFoundError(f"No .xlsx files found in {latest_year_dir!s}")

latest_report = max(xlsx_files, key=lambda p: p.stat().st_mtime)

report_path = latest_report
print(report_path)

dor = pd.read_excel(report_path, sheet_name="Summary - DC only", skiprows=6)


dor_end_date = report_path.stem.split(" - ")[-2]

dor_end_date = (
    pd.to_datetime(dor_end_date, format="%m%y", errors="raise")
    .to_period("M")
    .to_timestamp("M")
    .date()
)

print(dor_end_date)

J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Monthly Operating Statements\2025\Cumulative Report - Operating Statements - 1125 - Hard Coded.xlsx
2025-11-30


In [ ]:
# TRAC Data

trac_raw = pd.read_json(r"J:\TRAC\TRAC_Data.json", encoding="utf_16")

trac = pd.json_normalize(trac_raw["TRAC_Data"])

trac.index = trac_raw.index

s = trac["Agreement_CreatedDate"]
res = pd.to_datetime(s, format="%Y-%m-%dT%H:%M:%S.%f", errors="coerce")
res = res.fillna(pd.to_datetime(s, format="%Y-%m-%dT%H:%M:%S", errors="coerce"))
res = res.fillna(pd.to_datetime(s, errors="coerce"))

trac["Agreement_CreatedDate"] = res

In [ ]:
# Florence Data

In [ ]:
# Appian Data

appian = pd.read_excel(r"C:\Users\O304312\Downloads\SQL for Reporting v5.7.xlsx")